In [1]:
import os
import sys
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import pprint
import numpy as np
import random
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

output_dir = './local_run'
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)

def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100,
        init_conv_size = 4,
        g_channels = [128, 64, 32, 1],
        d_channels = [32, 64, 128, 256],
        batch_size = 128,
        learning_rate = 0.002,
        beta1 = 0.5,
        img_size = 32,
    )

hps = get_default_params()
class MnistData(object):
    def __init__(self, mnist_train, z_dim, img_size):
        self._data = mnist_train
        self._example_num = len(self._data)
        self._z_data = np.random.standard_normal((self._example_num, z_dim))
        self._indicator = 0
        self._resize_mnist_img(img_size)
        self._random_shuffle()
    
    def _random_shuffle(self):
        p = np.random.permutation(self._example_num)
        self._z_data = self._z_data[p]
        self._data = self._data[p]
    
    def _resize_mnist_img(self, img_size):
        data = np.asarray(self._data * 255, np.uint8)
        data = data.reshape((self._example_num, 1, 28, 28))
        data = data.transpose((0, 2, 3, 1))
        new_data = []
        for i in range(self._example_num):
            img = data[i].reshape((28, 28))
            img = Image.fromarray(img)
            img = img.resize((img_size, img_size))
            img = np.asarray(img)
            img = img.reshape((img_size, img_size, 1))
            new_data.append(img)
        new_data = np.asarray(new_data, dtype=np.float32)
        new_data = new_data / 127.5 - 1
        self._data = new_data
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._example_num:
            self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self._example_num
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_z = self._z_data[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_z

mnist_data = MnistData(mnist.train.images, hps.z_dim, hps.img_size)
batch_data, batch_z = mnist_data.next_batch(5)
print(batch_data.shape)
print(batch_z.shape)
def conv2d_transpose(inputs, out_channel, name, training, with_bn_relu=True):
    with tf.variable_scope(name):
        conv2d_trans = tf.layers.conv2d_transpose(inputs, 
                                                  out_channel,
                                                  [5, 5],
                                                  strides=(2,2),
                                                  padding='SAME')
        if with_bn_relu:
            bn = tf.layers.batch_normalization(conv2d_trans, training=training)
            relu = tf.nn.relu(bn)
            return relu
        else:
            return conv2d_trans

def conv2d(inputs, out_channel, name, training):
    def leaky_relu(x, leak = 0.2, name = ''):
        return tf.maximum(x, x * leak, name = name)
    with tf.variable_scope(name):
        conv2d_output = tf.layers.conv2d(inputs,
                                         out_channel,
                                         [5, 5],
                                         strides = (2, 2),
                                         padding = 'SAME')
        bn = tf.layers.batch_normalization(conv2d_output,
                                           training = training)
        return leaky_relu(bn, name = 'outputs')


class Generator(object):
    def __init__(self, channels, init_conv_size):
        assert len(channels) > 1
        self._channels = channels
        self._init_conv_size = init_conv_size
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('generator', reuse=self._reuse):
            with tf.variable_scope('inputs'):
                fc = tf.layers.dense(
                    inputs, 
                    self._channels[0] * self._init_conv_size * self._init_conv_size)
                conv0 = tf.reshape(fc, [-1, self._init_conv_size, self._init_conv_size, self._channels[0]])
                bn0 = tf.layers.batch_normalization(conv0, training=training)
                relu0 = tf.nn.relu(bn0)
            
            deconv_inputs = relu0
            # deconvolutions * 4
            for i in range(1, len(self._channels)):
                with_bn_relu = (i != len(self._channels) - 1)
                deconv_inputs = conv2d_transpose(deconv_inputs,
                                                 self._channels[i],
                                                 'deconv-%d' % i,
                                                 training,
                                                 with_bn_relu)
            img_inputs = deconv_inputs
            with tf.variable_scope('generate_imgs'):
                # imgs value scope: [-1, 1]
                imgs = tf.tanh(img_inputs, name='imgaes')
        self._reuse=True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
        return imgs
    
        
class Discriminator(object):
    def __init__(self, channels):
        self._channels = channels
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        
        conv_inputs = inputs
        with tf.variable_scope('discriminator', reuse = self._reuse):
            for i in range(len(self._channels)):
                conv_inputs = conv2d(conv_inputs,
                                     self._channels[i],
                                     'conv-%d' % i,
                                     training)
            fc_inputs = conv_inputs
            with tf.variable_scope('fc'):
                flatten = tf.layers.flatten(fc_inputs)
                logits = tf.layers.dense(flatten, 2, name="logits")
        self._reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
        return logits


class DCGAN(object):
    def __init__(self, hps):
        g_channels = hps.g_channels
        d_channels = hps.d_channels
        self._batch_size = hps.batch_size
        self._init_conv_size = hps.init_conv_size
        self._batch_size = hps.batch_size
        self._z_dim = hps.z_dim
        self._img_size = hps.img_size
        
        self._generator = Generator(g_channels, self._init_conv_size)
        self._discriminator = Discriminator(d_channels)
    
    def build(self):
        self._z_placholder = tf.placeholder(tf.float32, (self._batch_size, self._z_dim))
        self._img_placeholder = tf.placeholder(tf.float32, 
                                               (self._batch_size, self._img_size, self._img_size, 1))
        
        generated_imgs = self._generator(self._z_placholder, training = True)
        fake_img_logits = self._discriminator(generated_imgs, training = True)
        real_img_logits = self._discriminator(self._img_placeholder, training = True)
        
        loss_on_fake_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_fake_to_fake = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.zeros([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_real_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = real_img_logits))
        
        tf.add_to_collection('g_losses', loss_on_fake_to_real)
        tf.add_to_collection('d_losses', loss_on_fake_to_fake)
        tf.add_to_collection('d_losses', loss_on_real_to_real)
        
        
        loss = {
            'g': tf.add_n(tf.get_collection('g_losses'), name = 'total_g_loss'),
            'd': tf.add_n(tf.get_collection('d_losses'), name = 'total_d_loss')
        }
        
        return (self._z_placholder, self._img_placeholder, generated_imgs, loss)
    
    def build_train(self, losses, learning_rate, beta1):
        g_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        g_opt_op = g_opt.minimize(losses['g'], var_list = self._generator.variables)
        d_opt_op = d_opt.minimize(losses['d'], var_list = self._discriminator.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name = 'train')


dcgan = DCGAN(hps)
z_placeholder, img_placeholder, generated_imgs, losses = dcgan.build()
train_op = dcgan.build_train(losses, hps.learning_rate, hps.beta1)
def combine_and_show_imgs(batch_imgs, img_size, rows=8, cols=16):
    # batch_imgs: [batch_size, img_size, img_size, 1]
    result_big_img = []
    for i in range(rows):
        row_imgs = []
        for j in range(cols):
            img = batch_imgs[cols * i + j]
            img = img.reshape((img_size, img_size))
            img = (img + 1) * 127.5
            row_imgs.append(img)
        row_imgs = np.hstack(row_imgs)
        result_big_img.append(row_imgs)
    result_big_img = np.vstack(result_big_img)
    result_big_img = np.asarray(result_big_img, np.uint8)
    result_big_img = Image.fromarray(result_big_img)
    return result_big_img

    
    
init_op = tf.global_variables_initializer()
train_steps = 10000

with tf.Session() as sess:
    sess.run(init_op)
    for step in range(train_steps):
        batch_img, batch_z = mnist_data.next_batch(hps.batch_size)
        
        fetches = [train_op, losses['g'], losses['d']]
        should_sample = (step + 1) % 50 == 0
        if should_sample:
            fetches += [generated_imgs]
        out_values = sess.run(fetches,
                              feed_dict = {
                                  z_placeholder: batch_z,
                                  img_placeholder: batch_img
                              })
        _, g_loss_val, d_loss_val = out_values[0:3]
        logging.info('step: %d, g_loss: %4.3f, d_loss: %4.3f' % (step, g_loss_val, d_loss_val))
        if should_sample:
            gen_imgs_val = out_values[3]
            
            gen_img_path = os.path.join(output_dir, '%05d-gen.jpg' % (step + 1))
            gt_img_path = os.path.join(output_dir, '%05d-gt.jpg' % (step + 1))
            
            gen_img = combine_and_show_imgs(gen_imgs_val, hps.img_size)
            gt_img = combine_and_show_imgs(batch_img, hps.img_size)
            
            gen_img.save(gen_img_path)
            gt_img.save(gt_img_path)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(5, 32, 32, 1)
(5, 100)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Ins

INFO:tensorflow:step: 81, g_loss: 6.390, d_loss: 0.344
INFO:tensorflow:step: 82, g_loss: 2.450, d_loss: 0.120
INFO:tensorflow:step: 83, g_loss: 1.047, d_loss: 0.511
INFO:tensorflow:step: 84, g_loss: 2.498, d_loss: 0.125
INFO:tensorflow:step: 85, g_loss: 2.952, d_loss: 0.165
INFO:tensorflow:step: 86, g_loss: 2.102, d_loss: 0.227
INFO:tensorflow:step: 87, g_loss: 1.486, d_loss: 0.400
INFO:tensorflow:step: 88, g_loss: 1.302, d_loss: 0.710
INFO:tensorflow:step: 89, g_loss: 2.104, d_loss: 0.887
INFO:tensorflow:step: 90, g_loss: 0.255, d_loss: 1.689
INFO:tensorflow:step: 91, g_loss: 6.922, d_loss: 3.367
INFO:tensorflow:step: 92, g_loss: 3.024, d_loss: 0.415
INFO:tensorflow:step: 93, g_loss: 0.240, d_loss: 1.685
INFO:tensorflow:step: 94, g_loss: 2.757, d_loss: 0.320
INFO:tensorflow:step: 95, g_loss: 3.068, d_loss: 0.336
INFO:tensorflow:step: 96, g_loss: 1.213, d_loss: 0.497
INFO:tensorflow:step: 97, g_loss: 0.915, d_loss: 0.752
INFO:tensorflow:step: 98, g_loss: 3.420, d_loss: 1.357
INFO:tenso

INFO:tensorflow:step: 228, g_loss: 5.116, d_loss: 0.197
INFO:tensorflow:step: 229, g_loss: 2.029, d_loss: 0.202
INFO:tensorflow:step: 230, g_loss: 0.684, d_loss: 0.837
INFO:tensorflow:step: 231, g_loss: 5.023, d_loss: 0.248
INFO:tensorflow:step: 232, g_loss: 4.489, d_loss: 0.399
INFO:tensorflow:step: 233, g_loss: 0.728, d_loss: 0.865
INFO:tensorflow:step: 234, g_loss: 3.634, d_loss: 0.804
INFO:tensorflow:step: 235, g_loss: 0.686, d_loss: 0.908
INFO:tensorflow:step: 236, g_loss: 3.840, d_loss: 1.003
INFO:tensorflow:step: 237, g_loss: 0.704, d_loss: 0.832
INFO:tensorflow:step: 238, g_loss: 3.403, d_loss: 0.311
INFO:tensorflow:step: 239, g_loss: 2.369, d_loss: 0.432
INFO:tensorflow:step: 240, g_loss: 0.546, d_loss: 1.113
INFO:tensorflow:step: 241, g_loss: 5.034, d_loss: 1.150
INFO:tensorflow:step: 242, g_loss: 1.939, d_loss: 0.317
INFO:tensorflow:step: 243, g_loss: 1.423, d_loss: 0.391
INFO:tensorflow:step: 244, g_loss: 3.605, d_loss: 0.132
INFO:tensorflow:step: 245, g_loss: 3.205, d_loss

INFO:tensorflow:step: 375, g_loss: 3.693, d_loss: 0.349
INFO:tensorflow:step: 376, g_loss: 0.759, d_loss: 0.896
INFO:tensorflow:step: 377, g_loss: 3.433, d_loss: 0.469
INFO:tensorflow:step: 378, g_loss: 1.970, d_loss: 0.569
INFO:tensorflow:step: 379, g_loss: 0.830, d_loss: 0.916
INFO:tensorflow:step: 380, g_loss: 4.801, d_loss: 0.629
INFO:tensorflow:step: 381, g_loss: 3.012, d_loss: 0.255
INFO:tensorflow:step: 382, g_loss: 1.169, d_loss: 0.540
INFO:tensorflow:step: 383, g_loss: 2.940, d_loss: 0.361
INFO:tensorflow:step: 384, g_loss: 1.549, d_loss: 0.542
INFO:tensorflow:step: 385, g_loss: 4.068, d_loss: 0.584
INFO:tensorflow:step: 386, g_loss: 0.349, d_loss: 1.755
INFO:tensorflow:step: 387, g_loss: 8.209, d_loss: 3.346
INFO:tensorflow:step: 388, g_loss: 1.429, d_loss: 0.477
INFO:tensorflow:step: 389, g_loss: 1.481, d_loss: 0.471
INFO:tensorflow:step: 390, g_loss: 2.208, d_loss: 0.302
INFO:tensorflow:step: 391, g_loss: 2.488, d_loss: 0.650
INFO:tensorflow:step: 392, g_loss: 0.811, d_loss

INFO:tensorflow:step: 522, g_loss: 3.139, d_loss: 0.247
INFO:tensorflow:step: 523, g_loss: 2.425, d_loss: 0.217
INFO:tensorflow:step: 524, g_loss: 2.095, d_loss: 0.331
INFO:tensorflow:step: 525, g_loss: 0.941, d_loss: 0.730
INFO:tensorflow:step: 526, g_loss: 2.901, d_loss: 1.559
INFO:tensorflow:step: 527, g_loss: 0.317, d_loss: 1.722
INFO:tensorflow:step: 528, g_loss: 2.690, d_loss: 1.052
INFO:tensorflow:step: 529, g_loss: 1.531, d_loss: 0.764
INFO:tensorflow:step: 530, g_loss: 0.917, d_loss: 0.876
INFO:tensorflow:step: 531, g_loss: 2.543, d_loss: 0.394
INFO:tensorflow:step: 532, g_loss: 2.829, d_loss: 0.380
INFO:tensorflow:step: 533, g_loss: 1.278, d_loss: 0.570
INFO:tensorflow:step: 534, g_loss: 2.483, d_loss: 0.404
INFO:tensorflow:step: 535, g_loss: 1.841, d_loss: 0.383
INFO:tensorflow:step: 536, g_loss: 2.489, d_loss: 0.486
INFO:tensorflow:step: 537, g_loss: 0.949, d_loss: 0.953
INFO:tensorflow:step: 538, g_loss: 2.503, d_loss: 0.966
INFO:tensorflow:step: 539, g_loss: 1.086, d_loss

INFO:tensorflow:step: 669, g_loss: 1.045, d_loss: 0.716
INFO:tensorflow:step: 670, g_loss: 3.638, d_loss: 0.990
INFO:tensorflow:step: 671, g_loss: 0.326, d_loss: 1.595
INFO:tensorflow:step: 672, g_loss: 4.408, d_loss: 1.069
INFO:tensorflow:step: 673, g_loss: 1.678, d_loss: 0.460
INFO:tensorflow:step: 674, g_loss: 1.008, d_loss: 0.739
INFO:tensorflow:step: 675, g_loss: 2.896, d_loss: 0.785
INFO:tensorflow:step: 676, g_loss: 1.248, d_loss: 0.586
INFO:tensorflow:step: 677, g_loss: 2.645, d_loss: 0.308
INFO:tensorflow:step: 678, g_loss: 1.985, d_loss: 0.299
INFO:tensorflow:step: 679, g_loss: 2.304, d_loss: 0.318
INFO:tensorflow:step: 680, g_loss: 2.152, d_loss: 0.497
INFO:tensorflow:step: 681, g_loss: 1.151, d_loss: 0.687
INFO:tensorflow:step: 682, g_loss: 4.369, d_loss: 0.646
INFO:tensorflow:step: 683, g_loss: 0.763, d_loss: 0.990
INFO:tensorflow:step: 684, g_loss: 3.877, d_loss: 1.078
INFO:tensorflow:step: 685, g_loss: 0.594, d_loss: 1.264
INFO:tensorflow:step: 686, g_loss: 3.356, d_loss

INFO:tensorflow:step: 816, g_loss: 3.169, d_loss: 0.290
INFO:tensorflow:step: 817, g_loss: 2.355, d_loss: 0.442
INFO:tensorflow:step: 818, g_loss: 1.311, d_loss: 0.683
INFO:tensorflow:step: 819, g_loss: 1.328, d_loss: 0.642
INFO:tensorflow:step: 820, g_loss: 2.840, d_loss: 0.605
INFO:tensorflow:step: 821, g_loss: 1.037, d_loss: 0.691
INFO:tensorflow:step: 822, g_loss: 1.994, d_loss: 0.452
INFO:tensorflow:step: 823, g_loss: 2.136, d_loss: 0.641
INFO:tensorflow:step: 824, g_loss: 1.353, d_loss: 0.576
INFO:tensorflow:step: 825, g_loss: 2.258, d_loss: 0.470
INFO:tensorflow:step: 826, g_loss: 2.508, d_loss: 0.327
INFO:tensorflow:step: 827, g_loss: 1.246, d_loss: 0.620
INFO:tensorflow:step: 828, g_loss: 3.610, d_loss: 1.008
INFO:tensorflow:step: 829, g_loss: 0.279, d_loss: 2.088
INFO:tensorflow:step: 830, g_loss: 5.675, d_loss: 1.975
INFO:tensorflow:step: 831, g_loss: 1.193, d_loss: 0.674
INFO:tensorflow:step: 832, g_loss: 2.112, d_loss: 0.398
INFO:tensorflow:step: 833, g_loss: 2.014, d_loss

INFO:tensorflow:step: 963, g_loss: 1.722, d_loss: 0.501
INFO:tensorflow:step: 964, g_loss: 1.960, d_loss: 0.408
INFO:tensorflow:step: 965, g_loss: 2.057, d_loss: 0.417
INFO:tensorflow:step: 966, g_loss: 2.301, d_loss: 0.403
INFO:tensorflow:step: 967, g_loss: 1.591, d_loss: 0.516
INFO:tensorflow:step: 968, g_loss: 2.428, d_loss: 0.539
INFO:tensorflow:step: 969, g_loss: 1.177, d_loss: 0.667
INFO:tensorflow:step: 970, g_loss: 4.340, d_loss: 1.259
INFO:tensorflow:step: 971, g_loss: 0.238, d_loss: 1.998
INFO:tensorflow:step: 972, g_loss: 5.114, d_loss: 1.527
INFO:tensorflow:step: 973, g_loss: 1.546, d_loss: 0.431
INFO:tensorflow:step: 974, g_loss: 1.420, d_loss: 0.473
INFO:tensorflow:step: 975, g_loss: 2.888, d_loss: 0.409
INFO:tensorflow:step: 976, g_loss: 1.533, d_loss: 0.560
INFO:tensorflow:step: 977, g_loss: 1.710, d_loss: 0.483
INFO:tensorflow:step: 978, g_loss: 1.986, d_loss: 0.533
INFO:tensorflow:step: 979, g_loss: 1.567, d_loss: 0.625
INFO:tensorflow:step: 980, g_loss: 1.547, d_loss

INFO:tensorflow:step: 1108, g_loss: 1.853, d_loss: 0.850
INFO:tensorflow:step: 1109, g_loss: 0.559, d_loss: 1.160
INFO:tensorflow:step: 1110, g_loss: 4.489, d_loss: 1.000
INFO:tensorflow:step: 1111, g_loss: 1.582, d_loss: 0.448
INFO:tensorflow:step: 1112, g_loss: 1.760, d_loss: 0.442
INFO:tensorflow:step: 1113, g_loss: 2.197, d_loss: 0.601
INFO:tensorflow:step: 1114, g_loss: 2.381, d_loss: 0.439
INFO:tensorflow:step: 1115, g_loss: 1.142, d_loss: 0.689
INFO:tensorflow:step: 1116, g_loss: 2.938, d_loss: 0.948
INFO:tensorflow:step: 1117, g_loss: 0.906, d_loss: 0.932
INFO:tensorflow:step: 1118, g_loss: 4.571, d_loss: 1.068
INFO:tensorflow:step: 1119, g_loss: 0.951, d_loss: 0.767
INFO:tensorflow:step: 1120, g_loss: 2.625, d_loss: 0.479
INFO:tensorflow:step: 1121, g_loss: 2.859, d_loss: 0.377
INFO:tensorflow:step: 1122, g_loss: 1.172, d_loss: 0.590
INFO:tensorflow:step: 1123, g_loss: 2.893, d_loss: 0.663
INFO:tensorflow:step: 1124, g_loss: 1.380, d_loss: 0.496
INFO:tensorflow:step: 1125, g_l

KeyboardInterrupt: 